In [40]:
import argparse
import pulp as pl
import numpy as np
from utils import *
import utils1 as ut
import time
from math import floor,log

### Utils from jana's code

In [41]:

def read_dat_file(file_path, sorted = False, print_summary = False):
    with open(file_path, 'r') as file:
        # Read the first four lines
        m = int(file.readline().strip())  # First line
        n = int(file.readline().strip())  # Second line
        load_sizes = list(map(int, file.readline().strip().split()))  # Third line
        item_sizes = list(map(int, file.readline().strip().split()))  # Fourth line

        if sorted:
            load_sizes.sort()

        # Read the rest of the data into a list (if needed)
        data = []
        for line in file:
            data.append(list(map(int, line.strip().split())))

    if print_summary:
        print("############SUMMARY############")
        print(f"couriers num: {m}")
        print(f"num items: {n}")
        print(f"Load sizes: {load_sizes}")
        print(f"Item sizes: {item_sizes}")
        print("Distance matrix:")
        print(data)
        print("############END############\n")

    return m, n, load_sizes, item_sizes, np.array(data)

In [68]:
def process_instances_input(inst_path, res_path, solver_function):
    for filename in os.listdir(inst_path):
        inst_file = os.path.join(inst_path, filename)
        inst_id = int(filename[4:-4])
        if os.path.isfile(inst_file):
            print(f'\tCalculating results for instance {inst_file}')
            with open(inst_file,'r') as inst_file:
                i = 0
                for line in inst_file:
                    if i == 0:
                        n_couriers = int(line)
                    elif i == 1:
                        n_items = int(line)
                        dist_matrix = [None] * (n_items + 1)
                    elif i == 2:
                        capacity = [int(x) for x in line.split()]
                        assert len(capacity) == n_couriers
                    elif i == 3:
                        sizes = [int(x) for x in line.split()]
                        assert len(sizes) == n_items
                    else:
                        row = [int(x) for x in line.split()]
                        assert len(row) == n_items + 1
                        dist_matrix[i - 4] = [int(x) for x in row]
                    i += 1
            for i in range(len(dist_matrix)):
                assert dist_matrix[i][i] == 0
            print(f'\tLoaded input instance {inst_path}')
            result = solver_function(n_couriers, n_items, capacity, sizes, dist_matrix)
            #result need to be in format
            # result = {
            #     "time": exec_time,
            #     "optimal": optimal,
            #     "obj": obj if obj is not None else 0,  # Set to 0 if no objective found
            #     "sol": routes
            # }
            print(result)
            print(f'\tSolution computed')

            # Full path to the file
            file_path = os.path.join(res_path, f"{inst_id}.json")

            # Save the JSON data to the file
            with open(file_path, "w") as json_file:
                json.dump({"first_try": result}, json_file, indent=4)

            print(f"\tJSON data saved to {file_path}")

## My Model

In [77]:
class MIP_solver:

    __time_limit=0
    __instance=0

    #problem variables
    __m=0 #number of couriers
    __n=0 #number of items
    __D=[] #matrix of distances betweeen delivery points
    __L=[] #max load for each courier
    __S=[] #size of each item

    __n_loc=0 #number of locations in the problem
    __origin = 0 #index of the origin in the matrix 

    __solver =''
    __multiple_solvers=False

    #decision variables
    __lp_prob =''
    __x='' #3d array which is 1 if ...
    __courier_dist=''


    def __init__(self, solver=0, time_limit = 300, ) -> None:
        self.__time_limit=time_limit

        self.__lp_prob = LpProblem("Courier_Routing_Optimization", LpMinimize)
        if solver == 0:
            #in this case we'll try all the available solvers, instead of a chosen one
            self.__solver = pl.listSolvers()
            self.__multiple_solvers=True
        else:
            self.__solver=solver

        #self.run_model()
    
    def __configure_problem(self):

        #self.__lp_prob = LpProblem("Courier_Routing_Optimization", LpMinimize)

        num_cities = self.__D.shape[0] - 1
        depot = num_cities + 1
        max_items_per_courier = self.__n // self.__m + 1

        min_dist_bound = max(self.__D[num_cities, i] + self.__D[i, num_cities] for i in range(num_cities))
        lower_bound_dist = min(self.__D[num_cities, i] + self.__D[i, num_cities] for i in range(num_cities))

        dist_limit = min_dist_bound + 2 * round(log(sum(self.__D[i, i + 1] for i in range(max_items_per_courier + 1)))) + lower_bound_dist

        # Decision variables
        self.__x = LpVariable.dicts("route", (range(depot), range(depot), range(self.__m)), cat="Binary")
        node_vars = LpVariable.dicts("node", (range(num_cities), range(self.__m)), lowBound=0, upBound=depot - 1, cat="Integer")
        max_route_distance = LpVariable("max_route_distance", lowBound=min_dist_bound, upBound=dist_limit, cat="Integer")

        courier_weights = [
            LpVariable(f"courier_weight_{courier}", lowBound=0, upBound=self.__L[courier], cat="Integer")
            for courier in range(self.__m)
        ]

        self.__courier_dist = [
            LpVariable(f"courier_distance_{courier}", cat="Integer", lowBound=lower_bound_dist, upBound=dist_limit)
            for courier in range(self.__m)
        ]

        self.__lp_prob += max_route_distance

        # Weight constraints for each courier
        for courier in range(self.__m):
            self.__lp_prob += courier_weights[courier] == LpAffineExpression(
                [(self.__x[i][j][courier], self.__S[j]) for i in range(self.__n + 1) for j in range(self.__n)]
            )

        # Prevent self-looping arcs
        self.__lp_prob += lpSum(self.__x[i][i][courier] for i in range(depot) for courier in range(self.__m)) == 0

        # Each city visited exactly once
        for city in range(num_cities):
            self.__lp_prob += lpSum(self.__x[i][city][courier] for i in range(depot) for courier in range(self.__m)) == 1

        # Each courier departs from the depot
        for courier in range(self.__m):
            self.__lp_prob += lpSum(self.__x[num_cities][j][courier] for j in range(num_cities)) == 1

        # Each courier returns to the depot
        for courier in range(self.__m):
            self.__lp_prob += lpSum(self.__x[i][num_cities][courier] for i in range(num_cities)) == 1

        # Path connectivity constraints
        for city in range(depot):
            for courier in range(self.__m):
                self.__lp_prob += lpSum(self.__x[i][city][courier] for i in range(depot)) == lpSum(self.__x[city][i][courier] for i in range(depot))

        # Ensure no double usage of arcs
        for courier in range(self.__m):
            for i in range(num_cities):
                for j in range(num_cities):
                    self.__lp_prob += self.__x[i][j][courier] + self.__x[j][i][courier] <= 1

        # Subtour elimination constraints
        for courier in range(self.__m):
            for i in range(num_cities):
                for j in range(num_cities):
                    if i != j:
                        self.__lp_prob += node_vars[i][courier] - node_vars[j][courier] + num_cities * self.__x[i][j][courier] <= num_cities - 1

        # Calculate distances for each courier
        for courier in range(self.__m):
            self.__lp_prob += lpSum(self.__x[i][j][courier] * self.__D[i][j] for i in range(depot) for j in range(depot)) == self.__courier_dist[courier]

        # Max distance constraint
        for dist in self.__courier_dist:
            self.__lp_prob += max_route_distance >= dist



    
    def run_model_on_instance(self, instance):

        """
        Public method of the class. Reads the variables of the instance and starts the model
        """

        self.__instance=instance

        #building the file path from the instance number
        if self.__instance < 10:
            instances_path = "instances/inst0" + str(self.__instance) + ".dat"  #create the filepath
        else:
            instances_path = "instances/inst" + str(self.__instance) + ".dat"  #create the filepath


        #initialize all the problem variables
        self.__m, self.__n, self.__L,  self.__S, self.__D =  read_dat_file(instances_path)
            

        self.__configure_problem()
        
        self.__lp_prob.solve()
        status=self.__lp_prob.status
        print(f'++++++++++++++++++++++ \n{status}')
        if status == 1:
            solve_time= 300 if floor(self.__lp_prob.solutionTime) > 300 else floor(self.__lp_prob.solutionTime)
            opt=(self.__time_limit > solve_time)
            #print(f'++++++SolveTime:{self.__lp_prob.objective, solve_time}++++++')
            print(f'++++++++++++++++++++ \n OBJECTIVE VALUE: {value(self.__lp_prob.objective)} \n++++++++++++++++++++')
        else:
            print(solve_time)
            print('else')

        #TODO: manage multi-solver instance
            
    def get_x(self):
        return self.__x

In [78]:
solver = MIP_solver(solver='HiGHS')
solver.run_model_on_instance(instance=4, )

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/andreacristiano/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/06/25_1mnt97hq60zy5672y3qzw0000gn/T/2cda007bff1a4026aa5548f17a25849a-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/06/25_1mnt97hq60zy5672y3qzw0000gn/T/2cda007bff1a4026aa5548f17a25849a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1664 COLUMNS
At line 12244 RHS
At line 13904 BOUNDS
At line 14979 ENDATA
Problem MODEL has 1659 rows, 1065 columns and 8360 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 220 - 0.01 seconds
Cgl0002I 88 variables fixed
Cgl0004I processed model has 1218 rows, 969 columns (969 integer (880 of which binary)) and 7304 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0045I 1 integer variables out of 969 objects (969 

## Missing:

- Printing results to a json file
- linking utils to actual jana file
- putting file on docker


## Otto's model

In [21]:
def main(instance,configuration):
    time_limit = 300
    if configuration == 0:
        solvers = {
                    "CBC":PULP_CBC_CMD(timeLimit=time_limit),
                    "HiGHS":getSolver('HiGHS', timeLimit=time_limit,msg=False)
                }
    elif configuration == 1:
        solvers = {
                    "CBC":PULP_CBC_CMD(timeLimit=time_limit)
                }
    else:
        solvers = {
                    "HiGHS":getSolver('HiGHS', timeLimit=time_limit,msg=False)
                }
    if instance == 0:
        li = 1
        lu = 22
    else:
        li = instance
        print(li)
        lu = li+1
        
    for instance in range(li,lu):
        json_dict = {}
        n_couriers, n_items, courier_capacity,item_size, D = inputFile(instance)
        m_TSP,x,dis = set_const(n_couriers, n_items, courier_capacity,item_size, D)
        for solver in solvers:
            m_TSP.solve(solvers[solver])
            status = m_TSP.status
            if status == 1:    
                solve_time = 300 if floor(m_TSP.solutionTime) > 300 else floor(m_TSP.solutionTime)
                opt = (time_limit > solve_time)
                json_dict[solver] = jsonizer(x,n_items+1,n_couriers,solve_time,opt,value(m_TSP.objective))
            else:
                json_dict[solver] = jsonizer(x,n_items+1,n_couriers,300,False,-1)
        format_and_store(instance,json_dict)

In [14]:
def set_const(n_couriers, n_items, courier_capacity,item_size, D):
    m_TSP = LpProblem("Minimize_m_TSP",LpMinimize)

    n_cities=D.shape[0]-1
    origin=n_cities+1
    n_obj = n_items // n_couriers + 1


    min_dist = max([(D[n_cities,i] + D[i,n_cities]) for i in range(0,n_cities)])
    low_cour = min([(D[n_cities,i] + D[i,n_cities]) for i in range(0,n_cities)])

    
    max_dist = min_dist + 2*round(log(sum([D[i,i+1] for i in range(n_obj+1)]))) + low_cour
    #We define a 3d matrix of variables x[i][j][c] means that courier c has used node (i,j) to leave city i and reach city j
    x = LpVariable.dicts("x", (range(origin), range(origin), range(n_couriers)), cat="Binary")
    u = LpVariable.dicts("u", (range(n_cities), range(n_couriers)), lowBound=0, upBound=origin-1, cat="Integer")
    maximum = LpVariable("max_dist",lowBound=min_dist, upBound=max_dist, cat="Integer")
    weigths = [LpVariable(name=f'weigth_{i}', lowBound=0, upBound=courier_capacity[i], cat="Integer")
                   for i in range(n_couriers)]
    cour_dist = [
            LpVariable(name=f'obj_dist{i}', cat="Integer", lowBound=low_cour, upBound=max_dist)
            for i in range(n_couriers)]
    
    m_TSP += maximum

    # Set the weight carried by each courier
    for k in range(n_couriers):
            m_TSP += weigths[k] == LpAffineExpression([
                (x[i][j][k], item_size[j])
                for i in range(n_items+1)
                for j in range(n_items)])
            
    # Ensure that we dont use useless arcs 
    m_TSP += lpSum(x[i][i][c] for i in range(origin) for c in range(n_couriers)) == 0

    # Ensure that every city is reached by one and only one courier
    for j in range(n_cities) :
        m_TSP += lpSum(x[i][j][c] for i in range(origin) for c in range(n_couriers)) == 1

    # Ensure that every courier leaves the depot 
    for c in range(n_couriers) :
        m_TSP += lpSum(x[n_cities][j][c] for j in range(n_cities)) == 1

    # Ensure that every courier reach again the depot
    for c in range(n_couriers) :
        m_TSP += lpSum(x[i][n_cities][c] for i in range(n_cities)) == 1

    # Ensure that each courier path it's connected
    for j in range(origin):
        for c in range(n_couriers):
            m_TSP += lpSum(x[i][j][c] for i in range(origin) ) ==   lpSum(x[j][i][c] for i in range(origin) )

    for c in range(n_couriers):
        for i in range(n_cities):
            for j in range(n_cities):
                m_TSP += (x[i][j][c]   + x[j][i][c])  <= 1

    for j in range(n_couriers):
        m_TSP += lpSum(x[i][j][c] for i in range(origin) for c in range(n_couriers)) == 1

    # To eliminate subroutes
    for k in range(n_couriers):
        for i in range(n_cities):
            for j in range(n_cities):
                if i != j:
                    m_TSP += u[i][k] - u[j][k] + n_cities * x[i][j][k] <= n_cities - 1

    for c in range(n_couriers):
        m_TSP += lpSum( x[i][j][c] * D[i][j] for i in range(origin) for j in range(origin)) == cour_dist[c]
    
    for d in cour_dist:
            m_TSP += maximum >= d

    return m_TSP,x,cour_dist